### QA 
1. input molecules graph data √
2. input correct env variable √  - btw: art is what seawater?
3. connect these data ×
只匹配环境变量 20×16 = 320 和 1023对不上（这个因为同一环境条件下对一个分子不止做了一次实验，明天再研究下能怎么匹配），我明天再改下，
还是得从表格取env_df
卡住

### 1. Load molecules graph datas from pickle

In [2]:
## Load the molecular data (.pkl) for molecules with Targeted BDE & BDFE 

import os
import pickle
import networkx as nx
from torch_geometric.utils import from_networkx

folder_path = r"C:\Users\80710\OneDrive - Imperial College London\2025 engineering\GNN molecules\graph_pickles"

# Get all .pkl file paths in the folder
pkl_files = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith(".pkl")]

# Load all graphs
graph_list = []
for file_path in pkl_files:
    with open(file_path, "rb") as file:
        graph = pickle.load(file)  # Load each .pkl graph
        graph_list.append(graph)  # Store in a list

print(f"Loaded {len(graph_list)} graphs")


Loaded 20 graphs


In [3]:
## 检查
import pandas as pd

# Select the first graph for visualization
nx_graph = graph_list[0]

# Convert node data to DataFrame
node_data = pd.DataFrame.from_dict(dict(nx_graph.nodes(data=True)), orient="index")

# Convert edge data to DataFrame
edge_data = pd.DataFrame.from_records([(u, v, d) for u, v, d in nx_graph.edges(data=True)], 
                                      columns=["source", "target", "attributes"])

# Expand edge attributes
if not edge_data.empty and isinstance(edge_data.iloc[0]["attributes"], dict):
    attr_df = edge_data["attributes"].apply(pd.Series)  # Convert dict to columns
    edge_data = pd.concat([edge_data.drop(columns=["attributes"]), attr_df], axis=1)

# Display DataFrames
from IPython.display import display
display(node_data)
display(edge_data)



,symbol,rdkit_idx
0,C,0.0
1,C,1.0
2,C,2.0
3,C,3.0
4,C,4.0
5,C,5.0
6,C,6.0
7,C,7.0
8,C,8.0
9,C,9.0


,source,target,bond_index,bde_pred,bdfe_pred
0,0,1,0.0,89.382645,75.711845
1,0,H_23,25.0,100.077171,91.049126
2,1,2,1.0,85.872452,71.412849
3,1,H_27,29.0,97.163109,87.689636
4,2,3,2.0,85.041306,70.000282
5,2,H_28,30.0,95.392189,86.257263
6,3,4,3.0,83.115479,66.995277
7,3,H_30,32.0,94.518471,84.748619
8,4,5,NaN,NaN,NaN
9,4,10,NaN,NaN,NaN


Thinking: 为什么rdix里H对应的都是NAN 我们转化时用的是H-H 单H 键不需要 再进入rdix编号

备注：os.listdir... 列出文件夹中文件名称，迭代获取，保留.pkl,output list
     "rb"对口pkl二进制

In [4]:
## Convert Graphs to PyTorch Geometric Format for GNN model
# 方便训练和GNN计算，转化为PyG

import os
import pickle
import torch
import pandas as pd
from torch_geometric.data import Data
import networkx as nx

# Path to dataset folder (Update this)
folder_path = r"C:\Users\80710\OneDrive - Imperial College London\2025 engineering\GNN molecules\graph_pickles"

# Get all .pkl file paths
pkl_files = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith(".pkl")]

# List to store PyTorch Geometric graphs
pytorch_graphs = []

# **Function to handle missing hydrogen atom labels**
def clean_node_labels(node_data):
    """Replace NaN hydrogen labels with unique identifiers."""
    return {node: idx if not pd.isna(node) else f"H_{idx}" for idx, node in enumerate(node_data)}

# 分别提取节点和边的数据重构graph
# **Loop through each .pkl file and convert to PyG format**
for i, file_path in enumerate(pkl_files):
    with open(file_path, "rb") as file:
        nx_graph = pickle.load(file)  # Load NetworkX graph

        if isinstance(nx_graph, nx.Graph):
            # **Relabel nodes to avoid NaN values**
            mapping = clean_node_labels(nx_graph.nodes())
            nx_graph = nx.relabel_nodes(nx_graph, mapping)

            # **Convert Edges to PyG format (edge_index)**
            edge_index = torch.tensor(list(nx_graph.edges()), dtype=torch.long).T
            if edge_index.numel() == 0:  # Handle cases with no edges
                edge_index = torch.empty((2, 0), dtype=torch.long)

            # **Extract Node Features (Use Atomic Symbols)**
            first_node = next(iter(nx_graph.nodes), None)
            if first_node is not None and "symbol" in nx_graph.nodes[first_node]:
                node_features = torch.tensor(
                    [[1 if nx_graph.nodes[n]["symbol"] == "C" else 0] for n in nx_graph.nodes()],
                    dtype=torch.float
                )  # One-hot encoding Carbon (C) vs Hydrogen (H)
            else:
                node_features = torch.zeros((len(nx_graph.nodes()), 1), dtype=torch.float)  # Default: 1D zero feature

            # **Extract Edge Features (BDE, BDFE) - Handle NoneType**
            edge_features_list = []
            for u, v in nx_graph.edges():
                bde = nx_graph[u][v].get("bde_pred", 0.0)  # Replace None with 0.0
                bdfe = nx_graph[u][v].get("bdfe_pred", 0.0)  # Replace None with 0.0
                edge_features_list.append([bde if bde is not None else 0.0, bdfe if bdfe is not None else 0.0])

            edge_features = torch.tensor(edge_features_list, dtype=torch.float) if edge_features_list else None

            # **Create PyG Graph**
            pyg_graph = Data(x=node_features, edge_index=edge_index, edge_attr=edge_features)

            # **Store Processed Graph**
            pytorch_graphs.append(pyg_graph)
            print(f"Converted Graph {i+1}: {len(nx_graph.nodes)} nodes, {len(nx_graph.edges)} edges")

print(f"\n Total converted graphs: {len(pytorch_graphs)}")


Converted Graph 1: 42 nodes, 44 edges
Converted Graph 2: 43 nodes, 45 edges
Converted Graph 3: 52 nodes, 56 edges
Converted Graph 4: 54 nodes, 58 edges
Converted Graph 5: 56 nodes, 60 edges
Converted Graph 6: 56 nodes, 60 edges
Converted Graph 7: 58 nodes, 62 edges
Converted Graph 8: 58 nodes, 62 edges
Converted Graph 9: 60 nodes, 64 edges
Converted Graph 10: 60 nodes, 64 edges
Converted Graph 11: 62 nodes, 66 edges
Converted Graph 12: 62 nodes, 66 edges
Converted Graph 13: 46 nodes, 48 edges
Converted Graph 14: 48 nodes, 50 edges
Converted Graph 15: 52 nodes, 54 edges
Converted Graph 16: 52 nodes, 54 edges
Converted Graph 17: 53 nodes, 55 edges
Converted Graph 18: 53 nodes, 55 edges
Converted Graph 19: 48 nodes, 52 edges
Converted Graph 20: 48 nodes, 52 edges

 Total converted graphs: 20


In [5]:
print(pytorch_graphs[0])


Data(x=[42, 1], edge_index=[2, 44], edge_attr=[44, 2])


---

## 2. Load the environmental variable

In [14]:
import pandas as pd

# Load the environmental data from Excel
env_file = r"C:\Users\80710\OneDrive - Imperial College London\2025 engineering\GNN molecules\graph_pickles\dataset02.xlsx"
env_df = pd.read_excel(env_file, engine='openpyxl')

# Select only the relevant columns for the environment
env_columns = ["temperature", "seawater", "time", "concentration", "degradation_rate"]

# Ensure all columns exist in the dataset
env_df = env_df[env_columns].copy()

# Convert categorical "seawater" to numerical (if needed)
env_df["seawater"] = env_df["seawater"].map({"sea": 1, "art": 0})  # Map "sea" → 1, "art" → 0

# Drop rows with missing values
env_df = env_df.dropna().reset_index(drop=True)

# Check if it matches the number of graphs
print(f"Loaded {len(env_df)} environment rows")
print(env_df.head())


Loaded 1023 environment rows
   temperature  seawater  time  concentration  degradation_rate
0         35.6         1    30             70          0.670914
1         35.6         1    30             70          0.680071
2         35.6         1    30             70          0.655230
3         35.6         1    30             70          0.625193
4         35.6         1    30             70          0.605853


In [15]:
print(len(pytorch_graphs))  # Should be 20
print(len(env_df))          # Should also be 20

# If they don’t match, fix by truncating or aligning:
if len(env_df) > len(pytorch_graphs):
    env_df = env_df.iloc[:len(pytorch_graphs)].reset_index(drop=True)
elif len(env_df) < len(pytorch_graphs):
    print("Warning: Not enough environment rows, some graphs will be missing data.")


20
1023


---

## 3. Use environmental variables as Transformer Positional Encoding connect

simply use nx graph is enough

In [30]:
import torch
from torch_geometric.data import Data
from torch.utils.data import Dataset

class MoleculeEnvDataset(Dataset):
    def __init__(self, pyg_graphs, env_df):
        """
        pyg_graphs: list of PyG Data objects
        env_df: a pd.DataFrame with columns ["temperature", "seawater", "time", "concentration"], 
                or however you store your environment data.
        """
        self.pyg_graphs = pyg_graphs
        self.env_df = env_df  # DataFrame with shape [num_graphs, num_env_features]
        
        # Make sure lengths match
        assert len(self.pyg_graphs) == len(self.env_df), \
            "Number of graphs and env rows must match."
        
        # Pre-process environment features into numeric tensors
        # E.g., one-hot encode 'seawater' if it's categorical, etc.
        self.env_tensors = self.preprocess_env_features(self.env_df)

    def preprocess_env_features(self, df):
        """
        Convert 'sea'/'art' to 0 or 1, 
        or do a small one-hot if you prefer. 
        Also handle numeric columns. 
        Return a list of environment torch tensors.
        """
        env_tensors = []
        
        # Example: map sea -> 1, art -> 0
        sea_map = {'sea': 1.0, 'art': 0.0}
        
        for idx, row in df.iterrows():
            temperature = row["temperature"]       # float
            is_sea = sea_map.get(row["seawater"], 0.0)  # float
            time_val = row["time"]                # float
            concentration = row["concentration"]  # float
            
            # Build environment feature vector [T, sea/0-1, time, concentration]
            env_vec = torch.tensor([temperature, is_sea, time_val, concentration], dtype=torch.float)
            env_tensors.append(env_vec)
        
        return env_tensors

    def __len__(self):
        return len(self.pyg_graphs)

    def __getitem__(self, idx):
        """Return a (graph, environment_vector) tuple for each sample."""
        graph = self.pyg_graphs[idx]
        env_vec = self.env_tensors[idx]
        return graph, env_vec


In [31]:
import pandas as pd

# Suppose 'df_env' is your DataFrame loaded from dataset02.xlsx
# with columns [temperature, seawater, time, concentration] in the right order

dataset = MoleculeEnvDataset(pytorch_graphs, df_env)


AssertionError: Number of graphs and env rows must match.

### 3.1  建立Build DataLoader
PYG的Data对象代表图形，图形在节点和边缘的数量上有所不同。默认的DataLoader试图像张量一样堆叠它们，是不行滴。
 Custom Collate？

In [8]:
from torch_geometric.data import Batch
from torch.utils.data import DataLoader

def my_collate(batch):
    """
    batch is a list of tuples: (pyg_graph, env_features, target).
    We'll combine them into:
      - DataBatch of graphs
      - Stacked env_features
      - Stacked targets
    """
    graphs, envs, targets = zip(*batch)  # Unzip
    
    # 1) Batch the PyG graphs
    graphs_batch = Batch.from_data_list(graphs)
    
    # 2) Stack env features
    envs_batch = torch.stack(envs, dim=0)  # shape [batch_size, env_dim]
    
    # 3) Stack targets
    targets_batch = torch.stack(targets, dim=0)  # shape [batch_size]
    
    return graphs_batch, envs_batch, targets_batch


### 3.2 建立 env + molecules graph层 → 建立GNN
由于PYG将多个图批量分为一个大图结构，因此我们需要data.batch来确保每个节点都会收到其相应图的正确环境嵌入
1.PYG将多个图形组合在一起→来自不同图形的节点被连接
2.data.batch tracks nodes属于哪个图。
3.我们使用env_pos_emb[data.batch]添加每个节点的正确环境效果
4.这确保每个节点都会获取其相应的环境编码

In [32]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class EnvEmbeddingMLP(nn.Module):
    def __init__(self, env_dim=4, model_dim=128, hidden_dim=64):
        """
        env_dim: number of raw environment features
        model_dim: dimension of the final embedding to match your Transformer
        hidden_dim: dimension of the hidden layer in MLP
        """
        super().__init__()
        self.mlp = nn.Sequential(
            nn.Linear(env_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, model_dim)
        )
        
    def forward(self, env_vector):
        """
        env_vector: (batch_size, env_dim) or (env_dim,) if unbatched
        return: (batch_size, model_dim)
        """
        return self.mlp(env_vector)


In [33]:
class GraphTransformerModel(nn.Module):
    def __init__(self, 
                 node_feat_dim=1,      # e.g. your PyG node feature dimension
                 model_dim=128,
                 num_heads=4,
                 num_layers=3,
                 env_dim=4,
                 hidden_env=64):
        super().__init__()
        
        # Simple projection of node features up to model_dim
        self.node_proj = nn.Linear(node_feat_dim, model_dim)
        
        # MLP to produce environment-based embedding
        self.env_embed = EnvEmbeddingMLP(env_dim=env_dim, 
                                         model_dim=model_dim,
                                         hidden_dim=hidden_env)
        
        # Define a standard Transformer encoder
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=model_dim, 
            nhead=num_heads,
            dim_feedforward=256, 
            batch_first=True
        )
        self.transformer_encoder = nn.TransformerEncoder(
            encoder_layer, 
            num_layers=num_layers
        )
        
        # Output layer (example: regression for some property)
        self.out_layer = nn.Linear(model_dim, 1)
        
    def forward(self, graph_data, env_vector):
        """
        graph_data: a PyG Data object with:
            - x: node features shape [num_nodes, node_feat_dim]
        env_vector: shape [env_dim]
        
        Return: a single scalar or a node-level output, depending on design.
        """
        
        x = graph_data.x  # [num_nodes, node_feat_dim]
        
        # 1) Project node features to model_dim
        x = self.node_proj(x)  # [num_nodes, model_dim]
        
        # 2) Get environment embedding
        #    shape [model_dim] or [1, model_dim]
        env_emb = self.env_embed(env_vector.unsqueeze(0))  # -> [1, model_dim]
        
        # 3) Broadcast and add to every node
        #    if x has shape [N, model_dim], we want env_emb repeated N times
        x = x + env_emb  # Broadcasting: [N, model_dim]
        
        # 4) Transformer expects shape [batch_size, seq_len, d_model]
        #    We have 1 graph with N nodes => batch_size=1, seq_len=N
        x = x.unsqueeze(0)  # shape [1, N, model_dim]
        
        # 5) Pass through Transformer
        x = self.transformer_encoder(x)  # [1, N, model_dim]
        
        # 6) Pool or take first node, etc. (Your design choice)
        #    E.g., average pooling over node dimension
        x = x.mean(dim=1)  # [1, model_dim]
        
        # 7) Final output
        out = self.out_layer(x)  # [1, 1]
        return out


In [35]:
# ...
def forward(self, graph_data, env_vector):
    x = graph_data.x  # [num_nodes, node_feat_dim]
    x = self.node_proj(x)     # [num_nodes, model_dim]
    
    env_emb = self.env_embed(env_vector.unsqueeze(0))  # [1, model_dim]
    # Create a "batch" with shape [1, model_dim]. We'll treat it as an extra node
    x = torch.cat([env_emb, x], dim=0)  # shape [num_nodes+1, model_dim]
    
    x = x.unsqueeze(0)  # [1, num_nodes+1, model_dim]
    x = self.transformer_encoder(x)  # [1, num_nodes+1, model_dim]
    
    # For example, read out from the environment token (index 0):
    env_token_output = x[:, 0, :]  # shape [1, model_dim]
    out = self.out_layer(env_token_output)
    return out


### 3.3 MSE/RMSE/R2 检验 → 测试model

In [36]:
from torch.utils.data import DataLoader

def collate_fn(batch):
    """
    batch: list of (graph, env_vec) pairs
    We'll combine them if you want to do multi-graph batches. 
    For simplicity, let's do 1 graph per batch.
    """
    graphs, envs = zip(*batch)
    return graphs[0], envs[0]  # in a real scenario, you'd handle bigger batches

loader = DataLoader(dataset, batch_size=1, shuffle=True, collate_fn=collate_fn)

model = GraphTransformerModel(...)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

for epoch in range(10):
    for graph_data, env_vector in loader:
        optimizer.zero_grad()
        # Suppose you want to predict some property from your graph (like BDE)
        # We'll just do a random "target" for demonstration:
        target = torch.tensor([42.0])  # dummy
        pred = model(graph_data, env_vector)
        loss = (pred - target).abs().mean()  # L1 or MSE, etc.
        loss.backward()
        optimizer.step()
    
    print(f"Epoch {epoch} done.")


TypeError: empty(): argument 'size' failed to unpack the object at pos 2 with error "type must be tuple of ints,but got ellipsis"

---

In [12]:
# Suppose your environment DataFrame is called `df_env`.
# Suppose it has columns:
#  ["temperature", "seawater", "time", "concentration", "degradation_rate"]
# and each row matches one molecule in `pytorch_graphs`.

# 1. Create dataset
dataset = MoleculeEnvDataset(pyg_graphs=pytorch_graphs, env_df=df_env)

# 2. Create dataloader
from torch.utils.data import DataLoader
dataloader = DataLoader(dataset, batch_size=8, shuffle=True, collate_fn=my_collate)

# 3. Initialize model
num_node_features = 1  # if your x is [N,1], e.g. one-hot for C vs H
env_input_dim = 4       # [temperature, concentration, time, maybe 'seawater']
model = SimpleGraphModel(num_node_features=num_node_features,
                         env_input_dim=env_input_dim,
                         hidden_dim=128,
                         output_dim=1)

# 4. Train
train_model(model, dataloader, device='cpu', epochs=20)


AssertionError: Number of graphs must match number of rows in env_df!

In [11]:
train_model(model, dataloader, epochs=20)

NameError: name 'model' is not defined

## 4. fixed code

In [13]:
print(len(pytorch_graphs))   # Should be 20
print(len(env_df))           # Must also be 20


20


NameError: name 'env_df' is not defined

In [18]:
import torch
from torch_geometric.data import Data, Dataset
from torch.utils.data import DataLoader

class MoleculeEnvDataset(Dataset):
    def __init__(self, pyg_graphs, env_df):
        """
        Args:
            pyg_graphs: list of PyG Data objects (molecules)
            env_df: pandas DataFrame with environmental conditions & target
        """
        self.pyg_graphs = pyg_graphs
        self.env_df = env_df
        
        # 1. Expand molecule data to match env_df length
        self.graphs_expanded = [pyg_graphs[i % len(pyg_graphs)] for i in range(len(env_df))]

        # 2. Extract environmental conditions
        self.env_features = torch.tensor(
            self.env_df[["temperature", "seawater", "time", "concentration"]].values, dtype=torch.float
        )

        # 3. Extract degradation rate as target
        self.targets = torch.tensor(self.env_df["degradation_rate"].values, dtype=torch.float)

    def __len__(self):
        return len(self.env_df)  # Matches number of experimental conditions

    def __getitem__(self, idx):
        graph = self.graphs_expanded[idx]  # Get corresponding molecule
        env_features = self.env_features[idx]  # Get environmental variables
        target = self.targets[idx]  # Get degradation rate

        return graph, env_features, target


In [19]:
# 创建 Dataset
dataset = MoleculeEnvDataset(pyg_graphs=pytorch_graphs, env_df=env_df)

# 创建 DataLoader
dataloader = DataLoader(dataset, batch_size=8, shuffle=True)

# 检查 Dataset 结构
graph, env_features, target = dataset[0]
print(graph)  # Should be a PyG Data object
print(env_features)  # Should be a tensor with 4 environmental variables
print(target)  # Should be a single float value


Data(x=[42, 1], edge_index=[2, 44], edge_attr=[44, 2])
tensor([35.6000,  1.0000, 30.0000, 70.0000])
tensor(0.6709)


In [20]:
# 假设你的 GNN 结构如下：
num_node_features = 1  # 节点特征
env_input_dim = 4  # 环境变量：[温度，海水类型，时间，浓度]
model = SimpleGraphModel(num_node_features=num_node_features,
                         env_input_dim=env_input_dim,
                         hidden_dim=128,
                         output_dim=1)

# 训练模型
train_model(model, dataloader, device='cpu', epochs=20)


TypeError: default_collate: batch must contain tensors, numpy arrays, numbers, dicts or lists; found <class 'torch_geometric.data.data.Data'>

---

In [21]:
import itertools
import torch
import pandas as pd

# 所有环境变量的可能取值
temperature_values = [35.6, 86.0]
seawater_values = [0, 1]  # 这里用 0 (sea), 1 (art) 代替字符串
time_values = [30, 60]
concentration_values = [70, 700]

# **创建所有组合（16个环境变量组合）**
env_conditions = list(itertools.product(
    temperature_values, seawater_values, time_values, concentration_values
))

# 转为 Pandas DataFrame，便于查看
env_df = pd.DataFrame(env_conditions, columns=["temperature", "seawater", "time", "concentration"])
print(env_df)


    temperature  seawater  time  concentration
0          35.6         0    30             70
1          35.6         0    30            700
2          35.6         0    60             70
3          35.6         0    60            700
4          35.6         1    30             70
5          35.6         1    30            700
6          35.6         1    60             70
7          35.6         1    60            700
8          86.0         0    30             70
9          86.0         0    30            700
10         86.0         0    60             70
11         86.0         0    60            700
12         86.0         1    30             70
13         86.0         1    30            700
14         86.0         1    60             70
15         86.0         1    60            700


In [22]:
from torch_geometric.data import Dataset, Data, Batch
from torch.utils.data import DataLoader

class MoleculeEnvDataset(Dataset):
    def __init__(self, pyg_graphs, env_df):
        """
        Args:
            pyg_graphs: list of 20 PyG 分子图
            env_df: 16 组环境变量
        """
        self.pyg_graphs = pyg_graphs  # 20个分子
        self.env_df = env_df

        # **创建 (20 × 16) 组实验数据**
        self.dataset = [
            (self.pyg_graphs[i % len(self.pyg_graphs)], torch.tensor(env, dtype=torch.float))
            for i in range(len(self.pyg_graphs) * len(self.env_df))
            for env in self.env_df.values
        ]

    def __len__(self):
        return len(self.dataset)  # 320 组数据

    def __getitem__(self, idx):
        graph, env_features = self.dataset[idx]
        return graph, env_features

# **创建 Dataset**
dataset = MoleculeEnvDataset(pyg_graphs=pytorch_graphs, env_df=env_df)


In [25]:
class MoleculeEnvDataset(Dataset):
    def __init__(self, pyg_graphs, env_df, target_values):
        """
        Args:
            pyg_graphs: list of 20 PyG 分子图
            env_df: 16 组环境变量
            target_values: 320 组目标降解率 (每个分子在不同环境下的实验结果)
        """
        self.pyg_graphs = pyg_graphs
        self.env_df = env_df
        self.target_values = target_values  # 目标值 (320个)

        # **创建 (20 × 16) 组实验数据**
        self.dataset = [
            (self.pyg_graphs[i % len(self.pyg_graphs)],  # Graph
             torch.tensor(env, dtype=torch.float),       # 环境变量
             torch.tensor(self.target_values[i], dtype=torch.float))  # 目标值
            for i in range(len(self.pyg_graphs) * len(self.env_df))
            for env in self.env_df.values
        ]

    def __len__(self):
        return len(self.dataset)  # 320 组数据

    def __getitem__(self, idx):
        graph, env_features, target = self.dataset[idx]
        return graph, env_features, target


In [29]:
def my_collate(batch):
    graphs, env_features, targets = zip(*batch)  # 拆分 Graph, 环境变量, 目标值
    graphs_batch = Batch.from_data_list(graphs)  # PyG 批量处理
    env_features_batch = torch.stack(env_features)  # 转 Tensor
    targets_batch = torch.stack(targets)  # 转 Tensor
    return graphs_batch, env_features_batch, targets_batch

# **创建 DataLoader**
dataloader = DataLoader(dataset, batch_size=8, shuffle=True, collate_fn=my_collate)

# **检查 DataLoader**
graphs_batch, env_features_batch, targets_batch = next(iter(dataloader))
print(graphs_batch)  # PyG DataBatch
print(env_features_batch.shape)  # (batch_size, 4)
print(targets_batch.shape)  # (batch_size,)


ValueError: not enough values to unpack (expected 3, got 2)

In [28]:
# 假设 GNN 结构如下：
num_node_features = 1  # 节点特征
env_input_dim = 4  # 环境变量：[温度，海水类型，时间，浓度]

model = SimpleGraphModel(num_node_features=num_node_features,
                         env_input_dim=env_input_dim,
                         hidden_dim=128,
                         output_dim=1)

# **训练模型**
train_model(model, dataloader, device='cpu', epochs=20)


ValueError: not enough values to unpack (expected 3, got 2)